In [1]:
import pandas as pd


df = pd.read_json("ParlaSpeech-HR.v1.0.jsonl", orient="records", lines=True)
df = df.rename(columns={"path":"hashname"})

def process(text:str):
    from parse import compile
    from string import punctuation
    p = compile("{hit:d}.")
    in_list = text.split()
    out_list = list()
    for seg in in_list:
        parse_result = p.parse(seg)
        if parse_result:
            # We got a number with a dot afterward:
            out_list.append(seg.lower())
        else:
            out_list.append(seg.translate(str.maketrans('', '', punctuation)).lower())
    return " ".join(out_list)
df["sentence"] = df.words.apply(" ".join).apply(process)


In [2]:
df = pd.concat([df, df.speaker_info.apply(pd.Series)], axis=1)

In [3]:
test_df = df[df.split=="dev"].copy()

In [4]:
test_df.head(2)

,hashname,orig_file,start,end,words,word_start_times,norm_words,norm_words_start_times,utterance_id_start,utterance_id_end,...,norm_words_edited,sentence,Speaker_role,Speaker_type,Speaker_party,Speaker_party_name,Party_status,Speaker_name,Speaker_gender,Speaker_birth
1,Ki_SnDM_EkQ_2917.58-2937.5.wav,"18 10 2018 - 9. sjednica, 9. saziv [Ki_SnDM_Ek...",2917.58,2937.50,"[izraelska, tvrtka, prodaje, avione, Hrvatskoj...","[0, 0.65, 1.04, 1.54, 1.92, 2.51, 3.11, 3.5700...","[izraelska, tvrtka, prodaje, avione, hrvatskoj...","[0, 0.65, 1.04, 1.54, 1.92, 2.51, 3.11, 3.5700...",ParlaMint-HR_S09.u4267,ParlaMint-HR_S09.u4267,...,NaN,izraelska tvrtka prodaje avione hrvatskoj dire...,Regular,MP,"Živi zid, SNAGA",Klub Živog zida i SNAGA-e,Opposition,"Bunjac, Branimir",M,1972
2,XguZsDKdRh4_13797.59-13811.74.wav,"20 1 2017 - 3. sjednica, 9. saziv [XguZsDKdRh4...",13797.59,13811.74,"[jučer, trećem,, sutra, ili, prekosutra, četvr...","[0, 0.43, 1.0, 1.4, 1.52, 2.01, 2.55, 3.12, 3....","[jučer, trećem, sutra, ili, prekosutra, četvrt...","[0, 0.43, 1.0, 1.4, 1.52, 2.01, 2.55, 3.12, 3....",ParlaMint-HR_S03.u9702,ParlaMint-HR_S03.u9702,...,NaN,jučer trećem sutra ili prekosutra četvrtoj oso...,Regular,MP,HDZ,Klub Hrvatske demokratske zajednice,Coalition,"Bačić, Branko",M,1959


In [6]:
datadir = "/home/rupnik/macocu/task8/data_16000_mono"
transferdir = "/home/rupnik/macocu/task8/transfer"
import os
import shutil
for name in test_df.hashname.values:
    shutil.copy(
        os.path.join(
            datadir, "seg."+name
        ),
        os.path.join(transferdir, name)
        
        )

In [7]:
test_df.to_csv("35_eval_split.csv", index=False)

In [ ]:
l